## Hashing

At the foundation of any blockchain is the concept of hashing.

A hash function takes in data of any size and maps it to fixed-size values. You can think of these functions as "data-scramblers" that assign seemingly random and unique values to any data. I say *seemingly* because in reality, the values aren't actually random. However, for our purposes, we will be focusing on hash functions that utilize an extremely large pool of values to produce their output. And the size of this pool make it highly unlikely that the hash function will produce the same value for any two distinct inputs. However, when this does occur, it is called a *hash collision*. 

Technically, the output of a hash function is called a **digest**, although many refer to this output as simply the **hash**. In this article, I will use these terms interchangably. 

There are many different hash functions out there. For example, Bitcoin uses two consecutive rounds of sha256, while Ethereum uses keccak256. That being said, the important thing to remember is that these hash functions are different means to the same end--that is, they provide a predictable (but seemingly random) output for any input. 

The type of hash functions we are concerned with are cryptographic hash functions. Generally, these are hash functions that satisfy the following:
1. The same input will produce the same output
2. Given an output, it is infeasible to obtain the input (except by brute-force trial-and-error over a collasal number of iterations)
3. Given two unique inputs, the probability of a hash collision is basically zero
4. Any minor change in the input will produce vastly different results
5. Computationally fast to produce


To put things in perspective (in regards to point number 3 above), the probability of a hash collision for sha256 is 4.3^10-60. And Bitcoin uses two rounds of it.

As we’ve discussed, blockchains make use of cryptographic hash functions, which are hash functions that are extremely hard to reverse. Meaning, if we’re given a hash, it’s unfeasibly difficult (on a supercomputing level) to ever guess what the input was

In [37]:
import hashlib

data = "some text data"
bytes_data = bytes(data, "utf-8")
digest = hashlib.sha256(bytes_data).hexdigest()
digest


'613cda5e4af6b97997ed770e2c2a79571dee5a729632786e3b4b06a80d26e7f9'

Lets now add a single space to the end of our data and see how it changes the resulting hash:

In [40]:
data = "some text data "
bytes_data = bytes(data, "utf-8")
digest = hashlib.sha256(bytes_data).hexdigest()
digest

'33e67677240f8a63b142e66a35acb2f55877badd5566195ede5a8167db54d39d'

As you can see, even with the smallest of changes to the input data, the two resulting hashes look nothing alike
```
613cda5e4af6b97997ed770e2c2a79571dee5a729632786e3b4b06a80d26e7f9
33e67677240f8a63b142e66a35acb2f55877badd5566195ede5a8167db54d39d
```


Any type of data can be hashed. This is the premise behind NFTs. Lets look at an example.


In [151]:
from PIL import Image

# alter one pixel in the curiosity image
image_path = "curiosity_rover_selfie.png"

img = Image.open(image_path)
pixel_map = img.load()
pixel_map[0, 1] = (29, 30, 30, 255)  # originally was (30, 30, 30, 255)
img.save(f"altered_{image_path}")

# load and hash the original and altered image
with open(image_path, "rb") as f:
    original_image_hash = hashlib.sha256(f.read()).hexdigest()

with open(f"altered_{image_path}", "rb") as f:
    altered_image_hash = hashlib.sha256(f.read()).hexdigest()

# load and hash a screenshot of the original image
with open(f"curiosity_rover_selfie_screenshot.png", "rb") as f:
    screenshot_image_hash = hashlib.sha256(f.read()).hexdigest()

print("Original Image Hash Value:")
print(original_image_hash)

print("\nAltered Image Hash Value:")
print(altered_image_hash)

print("\nScreenshot Image Hash Value:")
print(screenshot_image_hash)



Original Image Hash Value:
dc611a2580bb313eb0a6194b2f713d1815a0a1b392c10e19e4e4c859e8326f2b

Altered Image Hash Value:
a27e344d3a3d329c45939561c5d40ff2f0d6b29573a6f5ddc049a1f2925e6e32

Screenshot Image Hash Value:
dc611a2580bb313eb0a6194b2f713d1815a0a1b392c10e19e4e4c859e8326f2b


## Proof-of-Work (POW)

The Proof-of-Work algorithm dates back to the early 90s and was originally used to prevent denial-of-service attacks. The idea behind the algorithm is straightforward: produce a hash value that satisfies some constraint.

In Bitcoin, the Proof-of-Work algorithm used is called Hashcash, and was developed by Adam Back in 1997 for email spam protection. 
The idea was to only accept emails that had a hash value below a certain number. This sounds pretty simple, but when considering that cryptographic hash functions produce a completely "random" number for a given input, one can start to see why this is effective. In fact, the only real way to satisfy this constraint is to insert arbitrary data, hash it, and then hope for the best. This process is done countless times until the value of the hash is less than the target number.

Lets simulate this. Below, I wrote a fake email and defined a target number equal to 1 million. In order for my email to be considered "not spam", the integer value of its hash must be less than this target number. To accomplish this, we iteratively append random values until this constraint is met. Note that the hash function used here is Shake-128, which lets you choose its output size.  

In [127]:
import random

target_number = 5_000_000

email = """
from: somebody123@gmail.com
subject: hash function concept demo

This is an email that, when hashed, must have a value less than the target number
in order to be accepted as valid.

Best,
Trevor

"""

num_iters = 0
r = False
while not r:
    r = int.from_bytes(hashlib.shake_128(bytes(email, "utf-8")).digest(4), "big") < target_number
    email += chr(random.choice(range(0, 128)))  # append a random character to the end of the email
    num_iters += 1

print(f"Number of Iterations to Meet The Constraint: {num_iters}\n")
print(email)


Number of Iterations to Meet The Constraint: 642


from: somebody123@gmail.com
subject: hash function concept demo

This is an email that, when hashed, must have a value less than the target number
in order to be accepted as valid.

Best,
Trevor

M4I(wpD40 ++N{E]G;-xKVrXDYDJu4p/Mhh"myU
aG"#Pu3*ze6<\Q|fh5F&3 mgXN*iIX~)p~I-? cH";??	y8#  ~_?lUcsNmm@4LwoHv=tC3D*J

:6}/yV fISpoV)R4!3OdU,]lf(BGY5g >M:kHb2!Cbq`RBP5wOKQg4KvQqqW
@^5<qTRX&EjKDB	~VfD-UX1TrWX
O1H%Av8x8]uqxxSakKW	8
@q+JQRX^]RA=	o9ljqhL^!:Z%&f5f(b$/#}yhf1v1%*~UCvDD4:pZ~A@qfw?=[b	z 
eSk-n$~azr8]Z7(4[7tp
2tHA@0W~ .,?V?.A% s)rrt O#~B e\%UPb6+6_]l6kGF A?:!0}u+W
R,-B(V(E%G
@^5<qTRX&EjKDB	~VfD-UX1TrWX
O1H%Av8x8]uqxxSakKW	8
@q+JQRX^]RA=	o9ljqhL^!:Z%&f5f(b$/#}yhf1v1%*~UCvDD4:pZ~A@qfw?=[b	z 
eSk-n$~azr8]Z7(4[7tp
2tHA@0W~ .,?V?.A% s)rrt O#~B e\%UPb6+6_]l6kGF A?:!0}u+W


It took 642 iterations of adding random characters to the end of the email until its hash had a value less than our target. This is the "work" in Proof-of-Work -- the computer must perform some operations to transform the desired data so that the hash can be verified as valid.



https://en.bitcoin.it/wiki/Secp256k1

## Blocks and the Blockchain

A so-called **block** can contain any type of data -- from images to financial transactions. Each block points to a previous block. For example, Block 100 contains the hash of Block 99, which contains the hash of Block 98, and so on. Stated differently, the blocks are *chained* together via their hashes -- hence the term **blockchain**. 

A simple block might look something like the below:

In [ ]:
{
    "index": 100,
    "timestamp": "2022-10-01T09:01:00",
    "data": [{"sender": "bob", "recipient": "alice", "amount": 500}],
    "prev_hash": "345dzxf253",
    "hash": "e9bcme42c7"
}


Notice how this block contains the field "prev_hash", which has a value equal to the hash of the previous block. This fact makes a blockchain immutable -- meaning it is impossible to change.

To show this, consider if an attacker manages to change a single bit of data in Block 99. If this occurs, the block's hash would change (remember that even the smallest change to a hash function's input will cause a wildly different result). This would then change the prev_hash field in Block 100, rendering it invalid. Ultimately, this would have a cascading effect for every single block after Block 99, causing the entire blockchain to become invalid. 

Stated differently, if someone attempted to commit fraud by altering the amount of a transaction, all subsequent hash values will change. This would immediately be recognized by everyone else in the network, and the change would be ignored by consensus. This is an important concept, as it serves as an underlying security mechanism behind a blockchain.

